In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0] #[0, 1, 2]
layer_sizes = [64] #[32, 64, 128]
conv_layers = [2] #[1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=3,
                      validation_split=0.3,
                      callbacks=[tensorboard])

2-conv-64-nodes-0-dense-1575229668
Train on 17462 samples, validate on 7484 samples
Epoch 1/3
17462/17462 [==============================] - 56s 3ms/sample - loss: 0.6227 - accuracy: 0.6447 - val_loss: 0.5500 - val_accuracy: 0.7219
Epoch 2/3
17462/17462 [==============================] - 59s 3ms/sample - loss: 0.5283 - accuracy: 0.7376 - val_loss: 0.5228 - val_accuracy: 0.7475
Epoch 3/3
17462/17462 [==============================] - 58s 3ms/sample - loss: 0.4912 - accuracy: 0.7668 - val_loss: 0.4877 - val_accuracy: 0.7670


In [9]:
model.save('to_save.model')

INFO:tensorflow:Assets written to: to_save.model/assets
